# Instructions
The following code was designed in order to track the location of a single animal across the course of a single video session.  After initally loading in the video, the user is able to crop the video frame using a rectangle selection tool.  A background reference frame is then specified, either by taking a median of several frames in the video, or by the user providing a short video of the same environment without an animal in it.  By comparing each frame in the video to the reference frame, the location of the animal can be tracked.  It is imperative that the reference frame of the video is not shifted from the actual video.  For this reason, if a separate video is supplied, it is best that it be acquired on the same day as the behavioral recording.  The animal's center of mass, in x,y coordinates, is then recorded, as well as the distance in pixels that the animal moves from one frame to the next. Lastly, the user can specify regions of interest in the frame (e.g. left, right) using a polygon drawing tool and record for each frame whether the animal is in the region of interest.  Options for summarizing the data are also provided. 

The notebook was updated by Davor Virag with some new functionality and usability improvements, which are documented in the notebook. Cells starting with `## CONFIG ##` indicate configurable options to be adjusted by the user, those starting with `## CODE ##` need not be edited, and `## INTERACTIVE ##` produce interactive elements for user input. These cells may begin with `%%output size = 100` or similar, which can be changed to adjust the element size.

---
# Imports
The following code loads neccessary packages and need not be changed by the user.

In [ ]:
## CODE ##
%load_ext autoreload
%autoreload 2
import os
import holoviews as hv
import numpy as np
import pandas as pd
import LocationTracking_Functions as lt
from pathlib import Path
import cv2
import pickle
import subprocess
import ipywidgets as widgets
from IPython.display import display, clear_output
import panel as pn

def create_layout(hv_obj):
    hv_pane = pn.pane.HoloViews(hv_obj)
    layout = pn.Column()
    layout.append(hv_pane)
    return layout

---
# Directory, file, and ROI settings

Whether to close a figure in the cell immediately after it is generated:

In [ ]:
## CONFIG ##
close_plots = False

### Paths and Configuration Persistence

The notebook now supports saving and loading your configuration settings (crop boundaries, ROIs, reference frame parameters, etc.) which enables three use cases:
1. **Process a single video with interactive setup**: Set `load_stored` to `False` (default) to go through the interactive setup steps for a new video.
2. **Use saved settings for interactive processing of multiple videos**: Set `load_stored` to `True` to load your saved configuration and then run the interactive setup cells to adjust parameters as needed for each new video. Same as before, but fewer work is needed as you start with your previous settings.
3. **Batch process multiple videos without interaction**: Set `load_stored` to `True` and generate a config file that you can apply to multiple videos using the batch processing `.py` scripts. The scripts allow you to generate multiple configs for multiple video groups, which is further explained in the scripts themselves.

Specifically, when **`load_stored`** is set to `True`, the notebook will attempt to load a previously saved `video_dict` configuration from disk. The `load_video_dict_storefile` will be loaded from the `dpath` directory. At the end of the notebook, the final `video_dict` of this session will be stored as `save_video_dict_storefile` - if it exists, it will be overwritten.

`dpath` is also the location where the videos will be loaded from for processing. Note that if you are using a Windows path with backslashes, place an ‘r’ in front of the directory path to avoid an error (e.g., `r"C:\Users\DeniseCaiLab\Videos"`).

In [ ]:
## CONFIG ##
load_stored = False
dpath = "/home/davor/ext/3/OF/"
dpath = Path(dpath)
load_video_dict_storefile = dpath/"video_dict_top_left.pickle"
#save_video_dict_storefile = None
save_video_dict_storefile = dpath/"video_dict_top_left.pickle"

In [ ]:
## CODE ##
if load_stored and load_video_dict_storefile.exists():
    with open(load_video_dict_storefile, 'rb') as f:
        video_dict = pickle.load(f)
else:
    video_dict = {}

### Main configuration

`file` : The filename of the video, including the file extension.

`start_s` : Seconds since the beginning of the video when to begin processing. 0 is the beginning, negative values can be used to specify a time relative to the *end* of the video. For example, `-600` will start processing at 5 minutes before the end of the video, and will process the last 5 minutes of the video (if `end_s` is `None`).

`end_s` : Seconds since the beginning of the video when to begin processing. If the user would like to process to the end of the video, this can be set to None.

`region_names` : If the user would like to measure the time spent in ROIs, a list containing the names of the ROIs should be provided.  A Python list is defined by a set of square brackets, and each ROI name should be placed in quotations, separated by a comma. If no ROIs are to be defined, this can be set to None (i.e., `‘region_names’ : None`).    
*(Note by davor: I haven't tested the updated version without any region names specified - please let me know if you do!)*

**Semi-automated region definition can be activated here** for OF and EPM. Simply specify either `"region_names": "OF"` or `"region_names": "EPM"` to use this functionality. It is described in more detail later on.

`dsmpl` : The amount to down-sample each frame. A value of 1 indicates no down-sampling, while a value of 0.25 indicates that each frame will be down-sampled to ¼ its original size.  Note that if down-sampling is performed, all pixel coordinate output will be in the dimensions of the down-sampled video.

`stretch` : Allows the user to alter the aspect ratio of the presented output. This is useful when videos have irregular dimensions and are difficult to see (e.g., an aspect ratio of 1:100). The width/height will be scaled by the factor provided. Note that this only affects the appearance of visualizations and does not modify the video or the interpretation of the output.

`angle` : A positive or negative decimal number specifying how many degrees to rotate the video. Can be handy when aligning pre-set OF and EPM ROIs which are strictly horizontal/vertical if the video has a slight rotation to it.

***Processing going slow?  Consider downsampling!***  Often times tracking does not not require 1080p or whatever high def resolution videos are sometimes acquired using. Try setting `dsmpl` to something lower than 1 to implement downsampling.

***Note:*** Options specified here will override any previously stored configuration when `load_stored` is set to `True`.

In [ ]:
## CONFIG ##
video_dict.update({
    'dpath'         : dpath,  # do not change here
    'file'          : "rotated_GX010486_top_left_upd.MP4",
    'start_s'       : -600, 
    'end_s'         : None,
    'region_names'  : ['wall_L','wall_R','wall_T','wall_B', 'corner_UL', 'corner_UR', 'corner_BL', 'corner_BR', 'center'],
#   'region_names'  : "OF",
#   'region_names'  : "EPM",
    'dsmpl'         : 0.5,
    'stretch'       : dict(width=1, height=1),
    'bins'          : None,
    'bin_duration_s': 60,
    'angle'         : None
})

In [ ]:
## CODE ##
video_dict["fpath"] = video_dict["dpath"]/video_dict["file"]
cap = cv2.VideoCapture(str(video_dict["fpath"]))

video_dict["fps"] = int(cap.get(cv2.CAP_PROP_FPS))
video_dict["start"] = int(video_dict["start_s"]*video_dict["fps"])
video_dict["first_frame"] = video_dict["start"]
video_dict["bin_duration"] = int(video_dict["bin_duration_s"]*video_dict["fps"])
video_dict["vid_duration"] = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
video_dict["end"] = int(video_dict["end_s"]*video_dict["fps"]) if video_dict["end_s"] is not None else int(cap.get(cv2.CAP_PROP_FRAME_COUNT))-1
if video_dict["start_s"] < 0:
    video_dict["start"] += video_dict["vid_duration"]
    video_dict["start_s"] += video_dict["vid_duration"]/video_dict["fps"]
if video_dict["end_s"] < 0:
    video_dict["end"] += video_dict["vid_duration"]
    video_dict["end_s"] += video_dict["vid_duration"]/video_dict["fps"]
if video_dict["start"] > video_dict["end"]:
    raise ValueError('ERROR: The specified start point is after the end point, please check video_dict["start_s"] and video_dict["end_s"]')

video_dict["fname_stem"] = Path(video_dict["file"]).stem
video_dict["output_path"] = video_dict["dpath"]/video_dict["fname_stem"]
video_dict["output_path"].mkdir(exist_ok=True)

cap.release()

When calculating a binned summary (i.e. stats per segment of video), bins can be specified in two ways.

**`bins_s`**: Custom bin starts and ends in seconds can be specified using the `bins_s` parameter. The timestamps are specified relative to the entire video, which means the first bin can start at the value of `start_s`, but not before. 

Negative values can be used, as well as `video_dict["start_s"]` and `video_dict["end_s"]` for the beginning and end of the video respectively, as well as arithmetic. No checks are implemented yet, so take care when specifying the values.

To illustrate many ways of specifying bins, an example is given with three bins specified. With the assumption that `video_dict["start_s"] = -600` and that the total video duration is `660 s`, each bin covers 10 seconds with a start at `video_dict["start_s"]` for a total of 30 seconds. The specification, in this case, is equivalent to:

```python
'bins_s': [
    (60, 70),
    (70, 80),
    (80, 90)
]
```

This parameter takes precedence over `bin_duration_s` (i.e. if `bins_s` is specified, `bin_duration_s` is ignored).

**`bin_duration_s`**: If `bins` is set to `None`, `bin_duration_s` is checked - the duration of each bin can be specified to generate the bins automatically.

If both parameters are set to `None`, a binned summary won't be calculated.

In [ ]:
## CONFIG ##
video_dict.update({
#   'bins_s'        : [
#       (video_dict["start_s"], video_dict["start_s"]+10),
#       (-590, -580),
#       (80, 90)
#   ],
    'bins_s'        : None,
    'bin_duration_s': 60
})

In [ ]:
if video_dict["bins_s"]:
    video_dict["bins"] = {}
    for i, b in enumerate(video_dict["bins_s"]):
        video_dict["bins"][str(i+1)] = (b[0]*video_dict["fps"], b[1]*video_dict["fps"])
elif video_dict["bin_duration"] and not video_dict["bins_s"]:
    video_dict["bins"] = {}
    for i, bin_start in enumerate(range(video_dict["start"], video_dict["vid_duration"], video_dict["bin_duration"])):
        video_dict["bins"][str(i+1)] = (bin_start, bin_start+video_dict["bin_duration"])
video_dict["last_frame"] = video_dict["vid_duration"]-video_dict["start"]
video_dict["full_bin"] = [(video_dict["start"], video_dict["last_frame"])]

### Video crop

To crop video frame, after running code below, select the box selection tool below the image (square with a plus sign). To start drawing region to be included in the analyis, double click image. Double click again to finalize region. If you decide to change region, it is best to rerun this cell and subsequent steps.

In [ ]:
%%output size = 300
## INTERACTIVE ##

#video_dict["crop"] = None
img_crp, crop_stream, video_dict = lt.LoadAndCrop(video_dict, cropmethod='Box')
video_dict["crop"] = crop_stream
#clear_output()
#img_crp, video_dict = lt.LoadAndCrop(video_dict, cropmethod=None)
#img_crp
layout = create_layout(img_crp)
layout

In [ ]:
## CODE ##
if close_plots:
    layout[-1] = pn.pane.Markdown("✅ Region captured and cleared.")

---
# Define reference frame for location tracking

For location tracking to work, a view of box without animal must be provided.  Below there are two ways to do this.  **Option 1** provides a method to remove the animal from the video.  This option works well provided the animal doesn't stay in the same location for >50% of the session. Alternatively, with **Option 2**, the user can simply define a video file in the same folder that doesn't have in animal in it.  Option 1 is generally recommended, being simpler to obtain. 

### Option 1 - Create reference frame by averaging the animal out of the video

The following code takes a random sample of frames across the session and creates an average of them by taking the median for each pixel.

The frames used can be specified using one of three arguments:
- `num_frames`: Specify the number of frames which are chosen at random
- `segment`: Specify a tuple, e.g. `(200,1000)`, to take the median out of frames from 200 to 1000.
- `frames`: Directly specify the frames to use. For example, NumPy can be used to specify a range `frames = np.arange(100,500,5)` would select every 5th frame in the range 100-500). Can be used as `segment`, where `frames=np.arange(200,1000)` does the same as `segment=(200,1000)`.

***Note by davor:*** The algorithm of obtaining frames from the video has been changed. Except in the case of very long videos (presumably, >1 hour), it is faster to go through the entire video frame-by-frame and store the ones we need for median calculation, than to actually skip ahead to the frame we need. This will still be fast if you specify a number of frames within e.g. 5 minutes of the video, since the algorithm will skip ahead, read frame-by-frame from the first to the last specified frame, and stop there. If you have a use case where you need to extract frames across the whole length of a long video and the algorithm is very slow, please let me know via GitHub or e-mail.

In [ ]:
## CONFIG ##

#video_dict['reference'], img_ref = lt.Reference(video_dict, num_frames=50) 
#video_dict['reference'], img_ref = lt.Reference(video_dict, frames=np.arange(100,2000,10))
video_dict['reference'], img_ref = lt.Reference(video_dict, segment=(200,1000))

In [ ]:
%%output size = 300
## INTERACTIVE ##

hv.save(img_ref, video_dict["output_path"]/str(video_dict["fname_stem"]+"_reference.png"), fmt='png')

layout = create_layout(img_ref)
layout

In [ ]:
## CODE ##
if close_plots:
    layout[-1] = pn.pane.Markdown("✅ Region captured and cleared.")

### Option 2 - Use a video of an empty box

The following code allows the user to specify a different file.  Set `video_dict['altfile']` to the alternative filename.  

Defining `frames` is necessary if the alternative video is a different length than the reference video.

In [ ]:
## CONFIG ##
%%output size = 100

video_dict['altfile'] = 'EmptyBox.avi' 

video_dict['reference'], img_ref = lt.Reference(video_dict, num_frames=50, altfile=True, frames=[0]) 
img_ref

---

# Define Regions of Interest (ROIs)


### Preset configuration

Here, you can specify configuration parameters if you're using the OF or EPM preset. If unclear on how the parameters impact the ROI sizes or placements, just try them out - the ROIs will be visualised a few cells below and you can re-run this entire segment with different parameters until the ROIs are satisfactory.

#### Open Field preset

In case of OF, it is assumed the arena spans the full area of the video frame. This can be achieved via careful cropping and by using the `video_dict["angle"]` parameter. `wall_fraction` defines how wide the wall region will be, expressed as a fraction of the frame width or height. By default, if `calculate_against` is `None`, horizontal wall-adjacent regions are calculated as a fraction of the frame's height, and vertical ones based on the frame width. This works well for square arenas in approximately square videos.

`calculate_against` can be set to `height` or `width`, to calculate the wall-adjacent region width as a fraction of the frame's height or width, respectively.

If the video is rectangular, the wall-adjacent ROIs won't be of the same width since they are calculated based on both  - let me know if you need a special preset for other arena shapes.

For example, if the frame is 400x450 (WxH) pixels, and `"wall_fraction": 0.2`, the animal will be counted as in the centre if its centre of mass is at least 80 px (20% of 400 px) away from the vertical walls, or 90 px away from the horizontal walls. If, for example, `calculate_against` is set to `height`, the value will be 90 px away from any of the walls.

In [ ]:
## CONFIG ##
OF_preset_config = {
    "wall_fraction": 0.2, # 20% of frame width/height
    "calculate_against": None
#   "calculate_against": "height"
#   "calculate_against": "width"
}

#### Elevated Plus Maze preset

In EPM, the full ROI area is again based off of the whole frame so cropping and angle adjustment in the previous steps serves to align the areas properly here as well.

The ROIs will be a plus sign at the centre of the frame, offset on the horizontal axis by `centre_offset_x` pixels (positive = right, negative = left) and on the vertical axis by `centre_y` pixels (positive = down, negative = up).

Its vertical and horizontal arms wide as specified by `vertical_arm_width_frac` and `horizontal_arm_height_frac`, as a fraction of the frame's width and height, respectively. The areas outside of the arms are masked so no changes in these areas are registered or tracked.

In [ ]:
## CONFIG ##
EPM_preset_config = {
    "centre_offset_x": 11, # 11 pixels to the right
    "centre_offset_y": -1, # 1 pixel up
    "vertical_arm_width_frac": 0.085, # 8.5% of frame width
    "horizontal_arm_height_frac": 0.085 # 8.5% of frame height
}

In [ ]:
## CODE ##
viewpane_dimensions = {"x": abs(video_dict["crop"].data["x1"][0]-video_dict["crop"].data["x0"][0]),
                       "y": abs(video_dict["crop"].data["y1"][0]-video_dict["crop"].data["y0"][0])}
max_x = viewpane_dimensions["x"]
max_y = viewpane_dimensions["y"]
if video_dict["region_names"] == "OF":
    roi_size = OF_preset_config["wall_fraction"]
    roi_width = roi_size*viewpane_dimensions["x"]
    roi_height = roi_size*viewpane_dimensions["y"]
    if OF_preset_config["calculate_against"] == "height":
        roi_width = roi_height
    elif OF_preset_config["calculate_against"] == "width":
        roi_height = roi_width
    #['wall_L','wall_R','wall_T','wall_B', 'corner_UL', 'corner_UR', 'corner_BL', 'corner_BR', 'center'],
    rois = { "wall_L" : {"xs": [0, 0, roi_width, roi_width],
                        "ys": [0, max_y, max_y, 0]},
            "wall_R" : {"xs": [max_x-roi_width, max_x-roi_width, max_x, max_x],
                        "ys": [0, max_y, max_y, 0]},
            "wall_T" : {"xs": [0, 0, max_x, max_x],
                        "ys": [0, roi_height, roi_height, 0]},
            "wall_B" : {"xs": [0, 0, max_x, max_x],
                        "ys": [max_y, max_y-roi_height, max_y-roi_height, max_y]},
            "corner_UL" : {"xs": [0, 0, roi_width, roi_width],
                            "ys": [0, roi_height, roi_height, 0]},
            "corner_UR" : {"xs": [max_x, max_x-roi_width, max_x-roi_width, max_x],
                            "ys": [0, 0, roi_height, roi_height]},
            "corner_BL" : {"xs": [0, roi_width, roi_width, 0],
                            "ys": [max_y, max_y, max_y-roi_height, max_y-roi_height]},
            "corner_BR" : {"xs": [max_x, max_x-roi_width, max_x-roi_width, max_x],
                            "ys": [max_y, max_y, max_y-roi_height, max_y-roi_height]},
            "center" : {"xs": [roi_width, max_x-roi_width, max_x-roi_width, roi_width],
                        "ys": [roi_height, roi_height, max_y-roi_height, max_y-roi_height]} }
    rois_poly = []
    for roi, coords in rois.items():
        rois_poly.append({("x", "y"):list(zip(coords["xs"], coords["ys"]))})
        #rois_poly.append({("xs", "ys"):[coords["xs"], coords["ys"]]})
    rois_poly = hv.Polygons(data=rois_poly)
    #print(rois_poly)
    roi_data = {"xs": [d["xs"] for d in rois.values()],
                "ys": [d["ys"] for d in rois.values()]}

    video_dict["roi_stream"] = lt.DataStub(roi_data)
elif video_dict["region_names"] == "EPM":
    centre_x = viewpane_dimensions["x"]/2+EPM_preset_config["centre_offset_x"]
    centre_y = viewpane_dimensions["y"]/2+EPM_preset_config["centre_offset_y"]
    roi_halfwidth = EPM_preset_config["vertical_arm_width_frac"]*viewpane_dimensions["x"]/2
    roi_halfheight = EPM_preset_config["horizontal_arm_height_frac"]*viewpane_dimensions["y"]/2
    #['open_T', 'open_B', 'closed_L', 'closed_R', 'center']
    rois = { "open_T" : {"xs": [centre_x-roi_halfwidth, centre_x-roi_halfwidth, centre_x+roi_halfwidth, centre_x+roi_halfwidth],
                        "ys": [0, centre_y-roi_halfheight, centre_y-roi_halfheight, 0]},
            "open_B" : {"xs": [centre_x-roi_halfwidth, centre_x-roi_halfwidth, centre_x+roi_halfwidth, centre_x+roi_halfwidth],
                        "ys": [max_y, centre_y+roi_halfheight, centre_y+roi_halfheight, max_y]},
            "closed_L" : {"xs": [0, centre_x-roi_halfwidth, centre_x-roi_halfwidth, 0],
                        "ys": [centre_y-roi_halfheight, centre_y-roi_halfheight, centre_y+roi_halfheight, centre_y+roi_halfheight]},
            "closed_R" : {"xs": [max_x, centre_x+roi_halfwidth, centre_x+roi_halfwidth, max_x],
                        "ys": [centre_y-roi_halfheight, centre_y-roi_halfheight, centre_y+roi_halfheight, centre_y+roi_halfheight]},
            "center" : {"xs": [centre_x-roi_halfwidth, centre_x-roi_halfwidth, centre_x+roi_halfwidth, centre_x+roi_halfwidth],
                        "ys": [centre_y-roi_halfheight, centre_y+roi_halfheight, centre_y+roi_halfheight, centre_y-roi_halfheight]}
    }
    mask = { "UL" : {"xs": [0, centre_x-roi_halfwidth, centre_x-roi_halfwidth, 0],
                    "ys": [0, 0, centre_y-roi_halfheight, centre_y-roi_halfheight]},
            "UR" : {"xs": [max_x, centre_x+roi_halfwidth, centre_x+roi_halfwidth, max_x],
                    "ys": [0, 0, centre_y-roi_halfheight, centre_y-roi_halfheight]},
            "BL" : {"xs": [0, centre_x-roi_halfwidth, centre_x-roi_halfwidth, 0],
                    "ys": [centre_y+roi_halfheight, centre_y+roi_halfheight, max_y, max_y]},
            "BR" : {"xs": [max_x, centre_x+roi_halfwidth, centre_x+roi_halfwidth, max_x],
                    "ys": [centre_y+roi_halfheight, centre_y+roi_halfheight, max_y, max_y]}
    }
    #rois_poly = []
    #for roi, coords in rois.items():
    #    rois_poly.append({("x", "y"):list(zip(coords["xs"], coords["ys"]))})
    #    #rois_poly.append({("xs", "ys"):[coords["xs"], coords["ys"]]})
    #rois_poly = hv.Polygons(data=rois_poly)
    #print(rois_poly)

    roi_data = {"xs": [d["xs"] for d in rois.values()],
                "ys": [d["ys"] for d in rois.values()]}
    video_dict["roi_stream"] = lt.DataStub(roi_data)

    mask_data = {"xs": [d["xs"] for d in mask.values()],
                "ys": [d["ys"] for d in mask.values()]}
    mask_bool = np.zeros(video_dict["f0"].shape)
    for submask in range(len(mask_data["xs"])):
        x = np.array(mask_data["xs"][submask]) #x coordinates
        y = np.array(mask_data["ys"][submask]) #y coordinates
        xy = np.column_stack((x,y)).astype("uint64") #xy coordinate pairs
        cv2.fillPoly(mask_bool, pts =[xy], color=1) #fill polygon
    mask_bool = mask_bool.astype("bool")
    video_dict["mask"] = {
        "stream": lt.DataStub(mask_data),
        "mask": mask_bool
    }

### Drawing/checking ROIs

After running cell below, draw regions of interest on presented image in the order you provided them (in Cell 2).  To start drawing a region, double click on image.  Single click to add a vertex.  Double click to close polygon.  If you mess up it's easiest to re-run cell.

***Note*** that there are no problems if regions overlap.

In [ ]:
%%output size = 300
## INTERACTIVE ##

img_roi, video_dict['roi_stream'] = lt.ROI_plot(video_dict)
hv.save(img_roi, video_dict["output_path"]/str(video_dict["fname_stem"]+"_ROIs.png"), fmt='png')
layout = create_layout(img_roi)
layout

In [ ]:
## CODE ##
if close_plots:
    layout[-1] = pn.pane.Markdown("✅ Region captured and cleared.")

### Mask internal regions



The following code is used to exclude internal portion/s of the field of view from the analysis. After running cell below, draw regions to be excluded.  To start drawing a region, double click on image.  Single click to add a vertex.  Double click to close polygon.  If you mess up it's easiest to re-run cell. 

If using the EPM preset, predefined masks should be shown outside of the maze's plus shape.

In [ ]:
%%output size = 90
## INTERACTIVE ##

#del(video_dict["mask"])
img_mask, video_dict['mask'] = lt.Mask_select(video_dict)
hv.save(img_mask, video_dict["output_path"]/str(video_dict["fname_stem"]+"_donottrack.png"), fmt='png')
layout = create_layout(img_mask)
layout

In [ ]:
## CODE ##
if close_plots:
    layout[-1] = pn.pane.Markdown("✅ Region captured and cleared.")

---


# (Optional) Define scale for distance calculations

### Option 1. Select two points of known distance

After running cell below, click on any two points and the distance between them, in pixel units, will be presented. Will be used to convert pixel distance to other scale. Note that once drawn, points can be dragged or you can click again. 

In [ ]:
%%output size = 300
## INTERACTIVE ##

img_scl, video_dict['scale'] = lt.DistanceTool(video_dict)
img_scl

### Option 2. Define real-world distance between points

Specify the translation between a distance in pixels to a real world distance. `px_distance` here is calculated as the diagonal between two opposite corners of the video frame as an example, but it can just as well be a number in pixels. I used this for a square OF arena 33x33 cm. Note that scale can be any desired text.

In [ ]:
## CONFIG ##
video_dict["scale"] = {
    "px_distance": np.sqrt(video_dict["reference"].shape[0]**2+video_dict["reference"].shape[1]**2),
    "true_distance": 47.376,
    "true_scale": "cm"
}

---


# Track location

### Set location tracking parameters
Location tracking examines the deviance of each frame in a video from the reference frame on a pixel by pixel basis.  For each frame it calculates the center of mass for these differences (COM) to define the center of the animal.  

`loc_thresh` : This parameter represents a percentile threshold and can take on values between 0-100.  Each frame is compared to the reference frame.  Then, to remove the influence of small fluctuations, any differences below a given percentile (relative to the maximum difference) are set to 0.  We use a value of 99.5 with good success.

`use_window` : This parameter is incredibly helpful if objects other than the animal temporarily enter the field of view during tracking (such as an experimenter’s hand manually delivering a stimulus or reward).  When use_window is set to True, a square window with the animal's position on the prior frame at its center is given more weight when searching for the animal’s location (because an animal presumably can't move far from one frame to the next).  In this way, the influence of objects entering the field of view can be avoided.  If use_window is set to True, the user should consider window_size and window_weight.

`window_size` : This parameter only impacts tracking when use_window is set to True.  This defines the size of the square window surrounding the animal that will be more heavily weighted in pixel units.  We typically set this to 2-3 times the animal’s size (if an animal is 100 pixels at its longest, we will set window_size to 200).  Note that to determine the size of the animal in pixels, the user can reference any image of the arena presented in ezTrack, which has the pixel coordinate scale on its axes.

`window_weight` : This parameter only impacts tracking when use_window is set to True.  When window_weight is set to 1, pixels outside of the window are not considered at all; at 0, they are given equal weight. Notably, setting a high value that is still not equal to 1 (e.g., 0.9) should allow ezTrack to more rapidly find the animal if, by chance, it moves out of the window.  

`method` : This parameter determines the luminosity of the object ezTrack will search for relative to the background and accepts values of 'abs', 'light', and 'dark'. Option 'abs' does not take into consideration whether the animal is lighter or darker than the background and will therefore track the animal across a wide range of backgrounds. 'light' assumes the animal is lighter than the background, and 'dark' assumes the animal is darker than the background. Option 'abs' generally works well, but there are situations in which you may wish to use the others.  For example, if a tether is being used that is opposite in color to the animal (a white wire and a black mouse), the ‘abs’ method is much more likely to be biased by the wire, whereas option ‘dark’ will look for the darker mouse.

`rmv_wire` : When rmv_wire is set to True, an algorithm is used to attempt to remove wires from the field of view.  If rmv_wire is set to True, the user should consider wire_krn.

`wire_krn` : This parameter only impacts tracking when rmv_wire is set to True. This value should be set between the width of the wire and the width of the animal, in pixel units. 

In [ ]:
## CONFIG ##
tracking_params = {
    'loc_thresh'    : 98.0, 
    'use_window'    : True, 
    'window_size'   : 150, 
    'window_weight' : .9, 
    'method'        : 'abs',
    'rmv_wire'      : True, 
    'wire_krn'      : 5,
    'progress_bar'  : True
}

### (Optional) Display examples of location tracking to confirm threshold
In order to confirm threshold is working, a subset of images is analyzed and displayed using the selected `tracking_params`. The original image is displayed on the left and the difference values to the right. The center of mass (COM) is pinpointed on images. Notably, because individual frames are used, window settings are not applicable here. Because of this, actual tracking in video is likely to be better.

The user can change the number examples below as they see fit.

In [ ]:
%%output size = 250
## INTERACTIVE ##

img_exmpls = lt.LocationThresh_View(video_dict, tracking_params, examples=16)
hv.save(img_exmpls, video_dict["output_path"]/str(video_dict["fname_stem"]+"_CheckTracking.png"), fmt='png')
layout = create_layout(img_exmpls.cols(4))
layout

In [ ]:
## CODE ##
if close_plots:
    layout[-1] = pn.pane.Markdown("✅ Region captured and cleared.")

### Save `video_dict` configuration to disk

The following code saves the current `video_dict` configuration to disk in two locations:
1. In the video's output directory (`video_dict["output_path"]/last_video_dict.pickle`), so each video has a reference of the `video_dict` config used to process it,
2. At the main data path (`save_video_dict_storefile`) for reuse across sessions.

This enables:
- Resuming work on the same video later without reconfiguring crop, ROIs, and reference frame
- Applying the saved configuration to new videos by setting `load_stored = True` at the beginning of the notebook
- Using the configuration file for batch processing with the `.py` scripts

The saved configuration includes all setup parameters: crop boundaries, ROIs, reference frame, tracking parameters, and scaling information.

Tracking parameters are not stored - if you need this, let me know.


In [ ]:
## CODE ##
if video_dict["region_names"] == "OF":
    video_dict["OF_preset_config"] = OF_preset_config.copy()
elif video_dict["region_names"] == "EPM":
    video_dict["EPM_preset_config"] = EPM_preset_config.copy()

with open(video_dict["output_path"]/"video_dict.pickle", 'wb') as f:
    pickle.dump(lt.copy_video_dict(video_dict), f)
if save_video_dict_storefile:
    with open(save_video_dict_storefile, 'wb') as f:
        pickle.dump(lt.copy_video_dict(video_dict), f)

### Track location and save results to a .csv file
For each frame the location of the animal's center of mass is recorded in x/y coordinates.  If ROIs are supplied, for each frame it is determined whether the animal is in each of the ROIs.  Frame-by-frame distance is also calculated in pixel units.  This data is returned in a Pandas dataframe with columns: frame, x, y, dist, and whether the animal is in each ROI specified (True/False).  Data is saved to a .csv in the same folder as the video.  First 5 rows of data are presented.

In [ ]:
## CODE ##
location = lt.TrackLocation(video_dict, tracking_params)   
#if __name__ == "__main__":
#    location = lt.TrackLocation_parallel(video_dict, tracking_params)
location.to_csv(video_dict["output_path"]/str(video_dict["fname_stem"]+'_LocationOutput.csv'), index=False)
location.head()

### (Optional) Display animal distance/location across session
Below, the animals distance and location across the video is plotted.  Smooth traces are expected in the case where the animal is tracked consistently across the session.  Under heatmap, sigma controls 'binning' of location. When `sigma = None` default value is provided; but sigma can also be set to any value. `width` and `height` control the size of the heatmap.

In [ ]:
## CONFIG ##
sigma = None
width = 150
height = 50 

In [ ]:
%%output size = 400
## INTERACTIVE ##

plt_dist = hv.Curve((location['Frame'],location['Distance_px']),'Frame','Pixel Distance').opts(
    height=h,width=w,color='red',title="Distance Across Session",toolbar="below")
plt_trks = lt.showtrace(video_dict, location, color="red", alpha=.05, size=2)
plt_hmap = lt.Heatmap(video_dict, location, sigma=sigma)
trace_img = (plt_trks + plt_hmap + plt_dist).cols(1)
hv.save(trace_img, video_dict["output_path"]/str(video_dict["fname_stem"]+"_TraceAndHeatmap.png"), fmt='png')
layout = create_layout(trace_img)
layout


In [ ]:
## CODE ##
if close_plots:
    layout[-1] = pn.pane.Markdown("✅ Region captured and cleared.")

---


# Create (binned) summary report and save


The code below creates binned and/or overall summary statistics for the session. The bins are defined in `video_dict` configuration at the beginning of the notebook.

In [ ]:
## CODE ##
if video_dict["bins"]:
    summary_binned = lt.Summarize_Location(location, video_dict, bin_dict=video_dict["bins"])
    summary_binned.to_csv(video_dict["output_path"]/str(video_dict["fname_stem"]+'_SummaryStats_binned.csv'), index=False)
summary = lt.Summarize_Location(location, video_dict, bin_dict=video_dict["full_bin"])
summary_full_filename = video_dict["dpath"]/str(video_dict["dpath"].stem+'_SummaryStats.csv')
if not summary_full_filename.exists():
    summary_full = summary
else:
    summary_full = pd.read_csv(summary_full_filename)
    if summary_full.loc[summary_full["File"] == video_dict["file"]].empty:
        summary_full = pd.concat([summary_full, summary])
    else:
        summary_full = summary_full.set_index("File")
        summary_full.update(summary.set_index("File"))
        summary_full = summary_full.reset_index()
summary_full.to_csv(summary_full_filename, index=False)

---


# (Optional) View video of tracking


**Note** that tracking must be done before this (Section [Track location](#track-location)). 

`start` : The frame video playback is to be started on. Note that this is relative to the start of tracking, where 0 is the first tracked frame.

`stop` : The frame video playback is to end on.  Note that this is relative to the start of tracking, where 0 is the first tracked frame.

`fps` : The speed of video playback.  Must be an integer.  Video playback may also be slower depending upon computer speed. 

`resize` : If the user wants the output to be larger or smaller, or they want the aspect ratio to be different, resize can be supplied as in the following example:
	`‘resize’ : (100,200)`
Here, the first number corresponds to the adjusted width of the frame, whereas the second number corresponds to the adjusted height.  Both numbers reflect pixel units and should be integers. Set resize equal to None if no resizing is to be done.

`save_video` : To save the video clip, set to True.


In [ ]:
## CONFIG ##
display_dict = {
    'start'      : 0, 
    'stop'       : video_dict["last_frame"]-video_dict["start"], 
    'fps'        : video_dict["fps"],
    'resize'     : None,
    'file'       : video_dict["fname_stem"]+"_tracked.mkv"
}

In [ ]:
## CODE ##
lt.SaveVideo(video_dict,display_dict,location)